In [1]:
# ===============================================
# STEP 1 — Build county → pixel mapping (one-time)
# ===============================================

import geopandas as gpd
import rasterio
import numpy as np
from rasterio.features import rasterize
from pathlib import Path
from tqdm import tqdm

COUNTY_FILE = r"D:/desktop/weather_data/geojson-counties-fips.json"   # 你的县边界文件
SAMPLE_TIF  = r"D:/desktop/weather_data/prism_raw/ppt/1990/ppt_19900101.tif"

OUT_MAP = r"D:/desktop/weather_data/county_to_idx.npy"

print("Loading county boundaries ...")
gdf = gpd.read_file(COUNTY_FILE)
gdf = gdf[~gdf["STATE"].isin(["02", "15", "72"])].copy()
gdf = gdf.to_crs("EPSG:4326")

gdf["id"] = gdf["id"].astype(int)

print("Reading sample PRISM raster ...")
with rasterio.open(SAMPLE_TIF) as src:
    height = src.height
    width  = src.width
    transform = src.transform

print("Rasterizing counties to PRISM grid ...")
shapes = [(geom, fips) for geom, fips in zip(gdf.geometry, gdf["id"])]

county_map = rasterize(
    shapes,
    out_shape=(height, width),
    transform=transform,
    fill=-1,
    dtype=np.int32
)

flat = county_map.reshape(-1)

print("Building sparse index for each county ...")
county_to_idx = {}
for fips in tqdm(gdf["id"]):
    county_to_idx[fips] = np.where(flat == fips)[0].astype(np.int32)

print("Saving mapping:", OUT_MAP)
np.save(OUT_MAP, county_to_idx, allow_pickle=True)
print("County → pixel mapping completed!")

Loading county boundaries ...
Reading sample PRISM raster ...
Rasterizing counties to PRISM grid ...
Building sparse index for each county ...


100%|██████████| 3109/3109 [00:00<00:00, 8660.14it/s]

Saving mapping: D:/desktop/weather_data/county_to_idx.npy
County → pixel mapping completed!


In [2]:
import cupy as cp
import numpy as np
import rasterio

def process_daily_tif(date_str, tif_paths, county_to_idx):
    """
    tif_paths = {"ppt": path1, "tmin": path2, "tmax": path3}
    return list of (date, fips, ppt, tmin, tmax)
    """
    gpu_arrays = {}

    # Load all 3 variables into GPU
    for element, tif_path in tif_paths.items():
        with rasterio.open(tif_path) as src:
            data = src.read(1).astype(np.float32)
        gpu_arrays[element] = cp.asarray(data.reshape(-1))

    rows = []

    # Loop counties
    for fips, idx in county_to_idx.items():
        gpu_idx = cp.asarray(idx)

        ppt  = float(cp.nanmean(gpu_arrays["ppt"][gpu_idx]).get())
        tmin = float(cp.nanmean(gpu_arrays["tmin"][gpu_idx]).get())
        tmax = float(cp.nanmean(gpu_arrays["tmax"][gpu_idx]).get())

        rows.append((date_str, fips, ppt, tmin, tmax))

    return rows

In [3]:
# ===============================================
# STEP 3 — Main pipeline: loop daily files (safe)
# ===============================================

import os
from pathlib import Path
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from datetime import date, timedelta
import numpy as np  # 别忘了 import
# from your_module import process_daily_tif  # 确保已经定义好了

RAW_DIR = Path(r"D:/desktop/weather_data/prism_raw")
OUT_DIR = Path(r"D:/desktop/weather_data")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# 总表（所有年份汇总）的路径（可选）
OUT_ALL = OUT_DIR / "prism_daily_county_all.csv"

# load county index
county_to_idx = np.load(
    r"D:/desktop/weather_data/county_to_idx.npy",
    allow_pickle=True
).item()
empty_fips = [fips for fips, idx in county_to_idx.items() if idx is None or len(idx) == 0]
print("Empty counties:", empty_fips)
# 过滤掉没有任何像素的县（len(idx) == 0）
county_to_idx = {
    fips: np.asarray(idx, dtype=np.int32)
    for fips, idx in county_to_idx.items()
    if idx is not None and len(idx) > 0
}
print("Counties kept after dropping empty ones:", len(county_to_idx))

ELEMENTS = ["ppt", "tmin", "tmax"]
YEARS = range(1990, 2025)

def find_tif(element, dt):
    return RAW_DIR / element / str(dt.year) / f"{element}_{dt:%Y%m%d}.tif"

def process_one_day(dt):
    """处理单天，返回若干 (date, fips, ppt, tmin, tmax)"""
    date_str = dt.strftime("%Y-%m-%d")

    paths = {}
    for e in ELEMENTS:
        p = find_tif(e, dt)
        if not p.exists():
            print("Missing:", p)
            return []
        paths[e] = p

    # GPU 计算
    return process_daily_tif(date_str, paths, county_to_idx)


def iter_dates_for_year(year):
    d = date(year, 1, 1)
    end = date(year, 12, 31)
    while d <= end:
        yield d
        d += timedelta(days=1)


def main():
    for y in YEARS:
        print(f"=== Processing year {y} ===")
        year_rows = []

        # 不用线程池，逐天顺序处理
        for d in iter_dates_for_year(y):
            rows = process_one_day(d)
            if rows:
                year_rows.extend(rows)

        if not year_rows:
            print(f"No data for year {y}, skip writing.")
            continue

        df_year = pd.DataFrame(year_rows, columns=["date","fips","ppt","tmin","tmax"])

        out_year_csv = OUT_DIR / f"prism_daily_county_{y}.csv"
        df_year.to_csv(out_year_csv, index=False)
        print("  Year written:", out_year_csv)

        del df_year
        del year_rows

    print("All years done!")

if __name__ == "__main__":
    main()

Empty counties: [51670, 51600, 51830, 51580, 51610, 51678, 51685, 51570]
Counties kept after dropping empty ones: 3101
=== Processing year 1990 ===
  Year written: D:\desktop\weather_data\prism_daily_county_1990.csv
=== Processing year 1991 ===
  Year written: D:\desktop\weather_data\prism_daily_county_1991.csv
=== Processing year 1992 ===
  Year written: D:\desktop\weather_data\prism_daily_county_1992.csv
=== Processing year 1993 ===
  Year written: D:\desktop\weather_data\prism_daily_county_1993.csv
=== Processing year 1994 ===
  Year written: D:\desktop\weather_data\prism_daily_county_1994.csv
=== Processing year 1995 ===
Missing: D:\desktop\weather_data\prism_raw\ppt\1995\ppt_19950101.tif
Missing: D:\desktop\weather_data\prism_raw\ppt\1995\ppt_19950102.tif
Missing: D:\desktop\weather_data\prism_raw\ppt\1995\ppt_19950103.tif
  Year written: D:\desktop\weather_data\prism_daily_county_1995.csv
=== Processing year 1996 ===
  Year written: D:\desktop\weather_data\prism_daily_county_1996